In [5]:
%pip install cloudinary

   ---------------------------------------- 0.0/147.8 kB ? eta -:--:--
   ---------------- ----------------------- 61.4/147.8 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 147.8/147.8 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cloudinary
import cloudinary.uploader
import os
import sys

# --- Cloudinary Configuration ---
# Replace with your actual Cloudinary credentials
# You can find these on your Cloudinary dashboard
CLOUD_NAME = 'dfecrk6wg'
API_KEY = '646246332894659'
API_SECRET = 'NisAj6_mUXo-HpE4YFR0oN6n-gk'

# Configure Cloudinary
cloudinary.config(
    cloud_name=CLOUD_NAME,
    api_key=API_KEY,
    api_secret=API_SECRET
)

# def upload_images_to_cloudinary(folder_path):
#     """
#     Uploads all image files from a specified folder to Cloudinary,
#     preserving their original filenames as public IDs.

#     Args:
#         folder_path (str): The path to the folder containing the images.
#     """
#     if not os.path.isdir(folder_path):
#         print(f"Error: Folder '{folder_path}' does not exist.", file=sys.stderr)
#         return

#     print(f"Starting upload from folder: {folder_path}")

#     # Supported image extensions (you can add more if needed)
#     image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp')

#     uploaded_count = 0
#     skipped_count = 0
#     error_count = 0

#     for filename in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, filename)

#         # Check if it's a file and has an image extension
#         if os.path.isfile(file_path) and filename.lower().endswith(image_extensions):
#             # Extract filename without extension to use as public_id
#             # Cloudinary will automatically add the correct extension based on the uploaded file
#             public_id = os.path.splitext(filename)[0]

#             try:
#                 print(f"Uploading '{filename}' with public_id '{public_id}'...")
#                 # The upload method
#                 # use_filename=True will instruct Cloudinary to use the original filename
#                 # unique_filename=False will prevent Cloudinary from adding random characters to the filename
#                 response = cloudinary.uploader.upload(
#                     file_path,
#                     public_id=public_id,
#                     overwrite=False, # Set to True if you want to replace existing images with the same public_id
#                     unique_filename=False # Important: Set to False to retain the original filename as public_id
#                 )
#                 print(f"Successfully uploaded '{filename}'. URL: {response['secure_url']}")
#                 uploaded_count += 1
#             except Exception as e:
#                 print(f"Error uploading '{filename}': {e}", file=sys.stderr)
#                 error_count += 1
#         else:
#             print(f"Skipping non-image file or directory: '{filename}'")
#             skipped_count += 1

#     print("\n--- Upload Summary ---")
#     print(f"Total files processed: {uploaded_count + skipped_count + error_count}")
#     print(f"Successfully uploaded: {uploaded_count}")
#     print(f"Skipped (non-image): {skipped_count}")
#     print(f"Failed uploads: {error_count}")
#     print("----------------------")

In [12]:
import openpyxl

In [ ]:
def upload_images_to_cloudinary_and_report(folder_path, cloudinary_target_folder, report_name):
    """
    Uploads all image files from a specified local folder to a target folder on Cloudinary,
    preserving their original filenames as public IDs.
    After successful uploads, it generates an XLSX report with image names and URLs.

    Args:
        folder_path (str): The path to the local folder containing the images.
        cloudinary_target_folder (str): The name of the folder within Cloudinary to upload to.
    """
    if not os.path.isdir(folder_path):
        print(f"Error: Local folder '{folder_path}' does not exist.", file=sys.stderr)
        return

    print(f"Starting upload from local folder: {folder_path}")
    print(f"Images will be uploaded to Cloudinary folder: '{cloudinary_target_folder}'")

    # Supported image extensions
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp')

    uploaded_images_data = [] # To store data for the XLSX report
    uploaded_count = 0
    skipped_count = 0
    error_count = 0

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a file and has an image extension
        if os.path.isfile(file_path) and filename.lower().endswith(image_extensions):
            # Extract filename without extension to use as public_id
            public_id = os.path.splitext(filename)[0]

            try:
                print(f"Uploading '{filename}' with public_id '{public_id}' to '{cloudinary_target_folder}'...")
                response = cloudinary.uploader.upload(
                    file_path,
                    public_id=public_id,
                    folder=cloudinary_target_folder, # This uploads to the specified folder in Cloudinary
                    overwrite=False, # Set to True if you want to replace existing images
                    unique_filename=False # Important: Set to False to retain the original filename as public_id
                )
                secure_url = response['secure_url']
                print(f"Successfully uploaded '{filename}'. URL: {secure_url}")
                uploaded_images_data.append({
                    'Image Name': public_id, # Image name without extension
                    'Image URL': secure_url
                })
                uploaded_count += 1
            except Exception as e:
                print(f"Error uploading '{filename}': {e}", file=sys.stderr)
                error_count += 1
        else:
            print(f"Skipping non-image file or directory: '{filename}'")
            skipped_count += 1

    print("\n--- Upload Summary ---")
    print(f"Total files processed: {uploaded_count + skipped_count + error_count}")
    print(f"Successfully uploaded: {uploaded_count}")
    print(f"Skipped (non-image): {skipped_count}")
    print(f"Failed uploads: {error_count}")
    print("----------------------")

    # --- Generate XLSX Report ---
    if uploaded_images_data:
        report_filename = report_name
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet.title = "Uploaded Images"

        # Write headers
        headers = ['Image Name', 'Image URL']
        sheet.append(headers)

        # Write data rows
        for item in uploaded_images_data:
            sheet.append([item['Image Name'], item['Image URL']])

        try:
            workbook.save(report_filename)
            print(f"\nSuccessfully generated XLSX report: {report_filename}")
        except Exception as e:
            print(f"Error saving XLSX report '{report_filename}': {e}", file=sys.stderr)
    else:
        print("\nNo images were successfully uploaded, so no XLSX report was generated.")

In [14]:
 # image_folder = './dealer_images' # Example: a folder named 'dealer_images' in the same directory

    # Option 2: Get folder path from command line argument
if len(sys.argv) > 1:
    image_folder = "./dealer_images"
else:
    print("Usage: python upload_script.py <path_to_your_image_folder>")
    print("Example: python upload_script.py ./my_dealer_images")
    # Fallback if no argument is provided
    image_folder = input("Please enter the path to your image folder: ")

cloudinary_folder_name = 'dealer_images'
report_name = './cloudinary_data/dealer_images.xlsx'
upload_images_to_cloudinary_and_report(image_folder, cloudinary_folder_name, report_name)

Starting upload from local folder: ./dealer_images
Images will be uploaded to Cloudinary folder: 'dealer_images'
Uploading 'D_ANJALITOY001.jpg' with public_id 'D_ANJALITOY001' to 'dealer_images'...
Successfully uploaded 'D_ANJALITOY001.jpg'. URL: https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276554/dealer_images/D_ANJALITOY001.jpg
Uploading 'D_ARUNTAT001.jpg' with public_id 'D_ARUNTAT001' to 'dealer_images'...
Successfully uploaded 'D_ARUNTAT001.jpg'. URL: https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276555/dealer_images/D_ARUNTAT001.jpg
Uploading 'D_ARUNTAT002.jpg' with public_id 'D_ARUNTAT002' to 'dealer_images'...
Successfully uploaded 'D_ARUNTAT002.jpg'. URL: https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276556/dealer_images/D_ARUNTAT002.jpg
Uploading 'D_ASHATAT001.jpg' with public_id 'D_ASHATAT001' to 'dealer_images'...
Successfully uploaded 'D_ASHATAT001.jpg'. URL: https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276557/dealer_images/D_ASHATAT001